In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from util_visualization import plot_feature_importances, plot_roc_curve
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_percentage_error, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve
from pylab import rcParams
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pickle
import importlib
import sys
import joblib
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import glob
import math

#For NN
from keras.models import Sequential
from tensorflow import keras
from keras import backend as K
import tensorflow_addons as tfa
from keras.layers import Dense, LSTM, BatchNormalization, RepeatVector, TimeDistributed
from keras.optimizers import Adam, RMSprop, Adagrad, Nadam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

## Import Data

In [2]:
#Import Dataset
labeled_index = pd.read_csv("../data/labeled_data/quarterly_labeled_features_standardized.csv")
labeled_index.set_index(labeled_index['date'], inplace=True)
quarters = np.sort(labeled_index.index.unique())

#Features Columns
features_columns = list(labeled_index.columns)
columns_to_remove = ['index', 'crash_label', 'date', 'Quarter']
features_columns = [column for column in features_columns if column not in columns_to_remove]
print(features_columns)

['volatility', 'price_change', 'volume_change', '000001.SS', '^BSESN', '^BVSP', '^FCHI', '^FTLC', '^GDAXI', '^GSPC', '^HSI', '^N225', '^SSMI', 'Crude_Oil_Index_Excess_Return', 'Ted_Rate', 'Gold_Price', 'housing', 'reserve', '10YGov_BondYield', 'current_acct', 'FX_Rate', 'turnover', 'Population', 'npl', 'Recession_Indicators', 'inflation', 'Unemployment', 'GDP']


## Split data

In [3]:
train_index = math.ceil(len(quarters) * .7)
val_index = math.ceil(len(quarters) * .85)
train_quarters, val_quarters, test_quarters = quarters[:train_index], quarters[train_index:val_index], quarters[val_index:]
train = labeled_index.loc[train_quarters]
val = labeled_index.loc[val_quarters]
test = labeled_index.loc[test_quarters]
X_train = train[features_columns]
y_train = train['crash_label']
X_val = val[features_columns]
y_val = val['crash_label']
X_test = test[features_columns]
y_test = test['crash_label']

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(620, 28) (620,) (130, 28) (130,) (130, 28) (130,)


## Oversampling

In [4]:
# Oversample the minority class (1) using SMOTE
oversampler = SMOTE(sampling_strategy=0.7)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
print(X_train_resampled.shape, y_train_resampled.shape)

(912, 28) (912,)


In [5]:
crash_count = y_train_resampled.value_counts()[1]
crash_percen = crash_count / y_train_resampled.shape[0]
print("Oversampled crash percentage: ", crash_percen)

Oversampled crash percentage:  0.41118421052631576


## Reshape

In [6]:
""" Changes data to the format for LSTM training 
for sliding window approach 
Goal: capture temporal dependencies in the data """
# Prepare the list for the transformed data

def lstm_transform(X_data, y_data, num_steps=5):
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(X_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= X_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = np.array(X_data)[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = np.array(y_data)[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_transformed = np.array(X)
    y_transformed = np.array(y)
    return x_transformed, y_transformed

In [7]:
X_train_reshaped, y_train_reshaped = lstm_transform(X_train_resampled, y_train_resampled)

X_val_reshaped, y_val_reshaped = lstm_transform(X_val, y_val)

X_test_reshaped, y_test_reshaped = lstm_transform(X_test, y_test)

In [8]:
print(X_train_reshaped.shape, y_train_reshaped.shape, X_val_reshaped.shape, y_val_reshaped.shape, X_test_reshaped.shape, y_test_reshaped.shape)

(907, 5, 28) (907,) (125, 5, 28) (125,) (125, 5, 28) (125,)


## Modelling

In [11]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

Metal device set to: Apple M1 Pro


2024-04-06 00:40:04.017615: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-06 00:40:04.018002: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [241]:
# Many-to-One Training
# X_train_reshaped[1]: number of time steps
# X_train_reshaped[2]: number of features

def create_model(optimizer='adam', learning_rate=0.001):
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = Adagrad(learning_rate=learning_rate)
    elif optimizer == 'nadam':
        opt = Nadam(learning_rate=learning_rate)
    else:
        raise ValueError('Invalid optimizer specified.')
        
    model = Sequential()
    # Bidirection?
    model.add(LSTM(64, dropout=0.2, input_shape=(X_train_reshaped.shape[1],X_train_reshaped.shape[2])))
    # model.add(Dense(64, activation='relu'))
    #model.add(Dropout(0.2))  # Adjust the dropout rate
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)
    
    return model

In [242]:
# Hyperparam Tuning: optimizer, learning rate, batch size
tscv = TimeSeriesSplit(n_splits=3)
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'adagrad', 'nadam'],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128]
}
lstm_model = KerasClassifier(build_fn=create_model)
grid_search = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=tscv)
grid_result = grid_search.fit(X_val_reshaped, y_val_reshaped)

# Get the best hyperparameters
best_learning_rate = grid_result.best_params_['learning_rate']
best_batch_size = grid_result.best_params_['batch_size']
best_optimizer = grid_result.best_params_['optimizer']

print("Best Optimizer:", best_optimizer)
print("Best Learning Rate:", best_learning_rate)
print("Best Batch Size:", best_batch_size)

2024-03-27 13:38:07.718573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 13:38:08.093403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_of

1/1 [==============================] - 2s 2s/step - loss: 0.7131 - auc: 0.0000e+00


2024-03-27 13:38:09.433633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 13:38:09.509108: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x31x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_of

1/1 [==============================] - 1s 888ms/step - loss: 0.7195 - auc: 0.0000e+00


2024-03-27 13:38:11.047292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 13:38:11.330317: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_of

...

1/1 [==============================] - 1s 1s/step - loss: 0.3167 - auc: 0.5000


2024-03-27 13:46:31.247417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 13:46:31.584720: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_of

3/4 [=====================>........] - ETA: 0s - loss: 0.6910 - auc: 0.7230

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x29x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x29x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

4/4 [==============================] - 5s 512ms/step - loss: 0.6825 - auc: 0.6443
Best Optimizer: adam
Best Learning Rate: 0.001
Best Batch Size: 32


In [243]:
model = create_model(best_optimizer, best_learning_rate)

model.summary()

Model: "sequential_182"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_182 (LSTM)             (None, 64)                23808     
                                                                 
 dense_180 (Dense)           (None, 1)                 65        
                                                                 
Total params: 23,873
Trainable params: 23,873
Non-trainable params: 0
_________________________________________________________________


In [260]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                classes=np.unique(y_train_reshaped), 
                y=y_train_reshaped)
train_class_weights = dict(enumerate(class_weights))
print(train_class_weights)

{0: 1.0028142589118199, 1: 0.9972014925373134}


In [261]:
model_result = model.fit(X_train_reshaped, y_train_reshaped, batch_size=best_batch_size, epochs=50, validation_data=(X_val_reshaped, y_val_reshaped), class_weight=train_class_weights)
loss, auc = model.evaluate(X_test_reshaped, y_test_reshaped)

Epoch 1/50


2024-03-27 15:09:08.523181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 15:09:10.033052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_of

33/34 [============================>.] - ETA: 0s - loss: 0.5517 - auc: 0.8645

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x13x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x13x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

34/34 [==============================] - ETA: 0s - loss: 0.5512 - auc: 0.8642

2024-03-27 15:09:17.682906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 15:09:17.779353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_of

34/34 [==============================] - 13s 213ms/step - loss: 0.5512 - auc: 0.8642 - val_loss: 0.8124 - val_auc: 0.8046
Epoch 2/50
34/34 [==============================] - 1s 39ms/step - loss: 0.3593 - auc: 0.9318 - val_loss: 0.6144 - val_auc: 0.7496
Epoch 3/50
34/34 [==============================] - 1s 33ms/step - loss: 0.3013 - auc: 0.9446 - val_loss: 0.5206 - val_auc: 0.7384
Epoch 4/50
34/34 [==============================] - 1s 28ms/step - loss: 0.2885 - auc: 0.9479 - val_loss: 0.5358 - val_auc: 0.7181
Epoch 5/50
34/34 [==============================] - 1s 25ms/step - loss: 0.2710 - auc: 0.9559 - val_loss: 0.5083 - val_auc: 0.7237
Epoch 6/50
34/34 [==============================] - 1s 25ms/step - loss: 0.2431 - auc: 0.9645 - val_loss: 0.4569 - val_auc: 0.6926
Epoch 7/50
34/34 [==============================] - 1s 27ms/step - loss: 0.2333 - auc: 0.9669 - val_loss: 0.4933 - val_auc: 0.6691
Epoch 8/50
34/34 [==============================] - 1s 25ms/step - loss: 0.2146 - auc: 0.970

In [262]:
y_prob = model.predict(X_test_reshaped)
y_pred = np.where(y_prob > 0.5, 1, 0) # Adjust the threshold as needed
accuracy = accuracy_score(y_test_reshaped.flatten(), y_pred.flatten())
rmse = np.sqrt(mean_squared_error(y_test_reshaped.flatten(), y_pred.flatten()))
mape = np.mean(np.abs((y_test_reshaped.flatten() - y_pred.flatten()) / y_test_reshaped.flatten())) * 100

conf_matrix = confusion_matrix(y_test_reshaped.flatten(), y_pred.flatten(), labels=[0, 1])

precision = precision_score(y_test_reshaped.flatten(), y_pred.flatten())
recall = recall_score(y_test_reshaped.flatten(), y_pred.flatten())
f1 = f1_score(y_test_reshaped.flatten(), y_pred.flatten())
fpr, tpr, thresholds = roc_curve(y_test_reshaped.flatten(), y_prob.flatten())

2024-03-27 15:13:23.286723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-03-27 15:13:23.339365: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 73ms/step


## Evaluation

In [263]:
evaluation_result = {
    'Train_Start': pd.to_datetime(train_quarters).min(),
    'Train_End': pd.to_datetime(train_quarters).max(),
    'Val_Start': pd.to_datetime(val_quarters).min(),
    'Val_End': pd.to_datetime(val_quarters).max(),
    'Test_Start': pd.to_datetime(test_quarters).min(),
    'Test_End': pd.to_datetime(test_quarters).max(),
    'Confusion_Matrix': conf_matrix, #[[TN, FP],[FN,TP]]
    'Precision': precision, 
    'Recall': recall, 
    'F1': f1, 
    'Accuracy': accuracy, 
    'RMSE': rmse,
    'MAPE': mape,
    'AUC-ROC': auc,
    'False Positive Rate': fpr,
    'True Positive Rate': tpr,
    'loss': loss
    }
evaluation_result

{'Train_Start': Timestamp('1998-03-31 00:00:00'),
 'Train_End': Timestamp('2013-06-30 00:00:00'),
 'Val_Start': Timestamp('2013-09-30 00:00:00'),
 'Val_End': Timestamp('2016-09-30 00:00:00'),
 'Test_Start': Timestamp('2016-12-31 00:00:00'),
 'Test_End': Timestamp('2019-12-31 00:00:00'),
 'Confusion_Matrix': array([[99, 25],
        [ 0,  1]]),
 'Precision': 0.038461538461538464,
 'Recall': 1.0,
 'F1': 0.07407407407407407,
 'Accuracy': 0.8,
 'RMSE': 0.4472135954999579,
 'MAPE': nan,
 'AUC-ROC': 0.8951612710952759,
 'False Positive Rate': array([0.        , 0.00806452, 0.10483871, 0.10483871, 1.        ]),
 'True Positive Rate': array([0., 0., 0., 1., 1.]),
 'loss': 1.0280542373657227}

In [ ]:
evaluation_df = pd.DataFrame(evaluation_result)
evaluation_df.to_csv('./evaluation_result/' + f"evaluation_LSTM_features.csv", index=False)
evaluation_df

TODO: https://stackoverflow.com/questions/77032200/how-can-i-correct-my-time-series-lstm-rnn-for-binary-classification-favoring-cla